In [ ]:
import pymongo
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
from bson import ObjectId

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["SA"]
collection = db["PRJ-16"]

In [ ]:
sensor_fields = ["S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
                 "S15", "S16", "S17", "S18", "S19", "S20",
                 "S21", "S22", "S23", "S24", "S25", "S26", "S27", "S28", "S29"
                 "T1", "T2", "T3", "T4", "T5", "T6"]

# sensor_fields = ["T5", "T6"]

pipeline = [
    {
        "$project": {
            **{s: {"$cond": [{"$gt": [f"${s}", None]}, 1, 0]} for s in sensor_fields}
        }
    },
    {
        "$group": {
            "_id": None,
            **{f"{s}_count": {"$sum": f"${s}"} for s in sensor_fields}
        }
    }
]

result = list(collection.aggregate(pipeline))[0]

In [ ]:
result

In [ ]:
from bson import ObjectId
import pandas as pd
import matplotlib.pyplot as plt

sensor = "S29"
value_field = "s"

pipeline = [
    {
        "$match": {
            f"{sensor}.{value_field}": {"$exists": True}
        }
    },
    {
        "$project": {
            "_id": 0,
            "time": {"$toDate": "$_id"},
            "value": f"${sensor}.{value_field}"
        }
    },
    {
        "$sort": {"time": 1}
    }
]

data = list(collection.aggregate(pipeline))
df_s29 = pd.DataFrame(data)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure datetime format
df_s29["time"] = pd.to_datetime(df_s29["time"])

# Extract year-month as string for grouping
df_s29["year_month"] = df_s29["time"].dt.to_period("M").astype(str)

# Loop over each month
for month, df_month in df_s29.groupby("year_month"):
    plt.figure(figsize=(12, 4))
    plt.scatter(df_month["time"], df_month["value"],
                s=8, color="#e65d19", alpha=0.7)

    plt.title(f"S29 – Tribúna horná – {month}")
    plt.xlabel("Čas (UTC)")
    plt.ylabel("Surová hodnota [μstrain]")
    plt.grid(True, linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
sensor_colors = {
    "S7": "#1f77b4",   "S8": "#ff7f0e",   "S9": "#2ca02c",   "S10": "#d62728",
    "S11": "#9467bd",  "S12": "#8c564b",  "S13": "#e377c2",  "S14": "#7f7f7f",
    "S15": "#bcbd22",  "S16": "#17becf",  "S17": "#aec7e8",  "S18": "#ffbb78",
    "S19": "#98df8a",  "S20": "#ff9896",  "S21": "#c5b0d5",  "S22": "#c49c94",
    "S23": "#f7b6d2",  "S24": "#c7c7c7",  "S25": "#dbdb8d",  "S26": "#9edae5",
    "S27": "#393b79",  "S28": "#637939",  "S29": "#e65d19"
}

In [ ]:
from bson import ObjectId
import pandas as pd
import matplotlib.pyplot as plt


def plot_monthly_sensor_data(collection, sensor_id, color):
    value_field = "s"

    pipeline = [
        {
            "$match": {
                f"{sensor_id}.{value_field}": {"$exists": True}
            }
        },
        {
            "$project": {
                "_id": 0,
                "time": {"$toDate": "$_id"},
                "value": f"${sensor_id}.{value_field}"
            }
        },
        {
            "$sort": {"time": 1}
        }
    ]

    data = list(collection.aggregate(pipeline))
    if not data:
        print(f"No data for {sensor_id}")
        return

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    df["year_month"] = df["time"].dt.to_period("M").astype(str)

    for month, df_month in df.groupby("year_month"):
        plt.figure(figsize=(12, 4))
        plt.scatter(df_month["time"], df_month["value"],
                    s=8, color=color, alpha=0.7)

        plt.title(f"{sensor_id} – {month}")
        plt.xlabel("Čas (UTC)")
        plt.ylabel("Surová hodnota [μstrain]")
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.tight_layout()
        plt.show()

In [ ]:
for sensor, color in sensor_colors.items():
    plot_monthly_sensor_data(collection, sensor, color)